In [1]:
# Alternate GUI for IAFautoclassification script using widgets from Jupyter
# Written by: Robert Granat, Jan-Feb 2022.

# Imports
import ipywidgets as widgets
import os
import src.IAFautoclass.SqlHelper.IAFSqlHelper as sql
import src.IAFautoclass.IAFautomaticClassiphyer

# Constants
DEFAULT_HOST = "tcp:sql-stat.iaf.local"
DEFAULT_ODBC_DRIVER = "SQL Server"
DEFAULT_CATALOG = "Arbetsdatabas"

In [2]:
# Welcome message
widgets.Label(
    value = "*** Welcome to IAF automatic classification! ***"
)

Label(value='*** Welcome to IAF automatic classification! ***')

In [3]:
# Project name
project = widgets.Text(
    value='',
    placeholder='Project name',
    description='Project name:',
    disabled=False
)
display(project)

Text(value='', description='Project name:', placeholder='Project name')

In [9]:
conn = None
if project.value != '':
    sqlHelper = sql.IAFSqlHelper(driver = DEFAULT_ODBC_DRIVER, host = DEFAULT_HOST, catalog = DEFAULT_CATALOG, \
                                 trusted_connection = True )
    try:
        conn = sqlHelper.connect()
    except Exception as ex:
        print("Could not connect to SQL Server: {str(ex)}")
    print("Project name: {}".format(project.value))
    print("Successfully connected to SQL Server!")
else:
    print("Pick a project name to move on!")

Project name: iris
Successfully connected to SQL Server!


In [10]:
# Ask for databases to build database dropdown list
if conn:
    query = "SELECT name FROM sys.databases"
    sqlHelper.execute_query(query, get_data = True)
    data = sqlHelper.read_all_data()
    
    database_list = [''] + [database[0] for database in data]
    database_dropdown = widgets.Dropdown(
        options=database_list,
        value=database_list[0],
        description='Databases:',
    )
    display(database_dropdown)

Dropdown(description='Databases:', options=('', 'master', 'tempdb', 'model', 'msdb', 'AF', 'astat3_gallring', …

In [11]:
if conn and database_dropdown.value != '':
    query = "SELECT TABLE_SCHEMA,TABLE_NAME FROM INFORMATION_SCHEMA.TABLES ORDER BY TABLE_SCHEMA,TABLE_NAME"
    sqlHelper.execute_query(query, get_data = True)
    tables = sqlHelper.read_all_data()
    
    tables_list = [''] + [table[0]+'.'+table[1] for table in tables]
    tables_dropdown = widgets.Dropdown(
        options=tables_list,
        value=tables_list[0],
        description='Tables:',
    )
    display(tables_dropdown)

Dropdown(description='Tables:', options=('', 'ar_hpl.ArendegranskningSvar', 'ar_hpl.EstimeringSvar', 'ar_hpl.I…

In [16]:
# Give directions 
if conn and tables_dropdown.value != '':
    widgets.Label(
        value = "*** Pick pretrained model or train a new one ***"
    )
    path = ".\\src\\IAFautoclass\\model\\"
    models_list = [''] + ['Train new model'] + [file for file in os.listdir(path)]
    models_dropdown = widgets.Dropdown(
        options=models_list,
        value=models_list[0],
        description='Models:',
    )
    display(models_dropdown)

Dropdown(description='Models:', options=('', 'Train new model', 'iris.sav'), value='')

In [20]:
if conn and models_dropdown.value == 'Train new model':
    query = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_CATALOG = " \
        + "\'" + database_dropdown.value + "\' AND CONCAT(CONCAT(TABLE_SCHEMA,'.'),TABLE_NAME) = " \
        + "\'" + tables_dropdown.value + "\'"
    sqlHelper.execute_query(query, get_data = True)
    columns = sqlHelper.read_all_data()

    columns_list = [column[0] for column in columns]
    
    class_column = widgets.RadioButtons(
        options=columns_list,
        value=columns_list[0],
        description='Class:',
        disabled=False
    )
    display(class_column)
    
    id_column = widgets.RadioButtons(
        options=columns_list,
        value=columns_list[1],
        description='Id:',
        disabled=False
    )
    display(id_column)
    
    data_columns = widgets.SelectMultiple(
        options=columns_list,
        value=columns_list[2:],
        description='Data:',
        disabled=False
    )
    display(data_columns)
elif conn and models_dropdown.value != '':
    print("Using model {0} on SQL table: {1}.{2}!".
         format(models_dropdown.value, database_dropdown.value, 
               tables_dropdown.value))
elif conn:
    print("Pick a model to move on!")

RadioButtons(description='Class:', options=('class', 'id', 'petal-length', 'petal-width', 'sepal-length', 'sep…

RadioButtons(description='Id:', index=1, options=('class', 'id', 'petal-length', 'petal-width', 'sepal-length'…

SelectMultiple(description='Data:', index=(2, 3, 4, 5), options=('class', 'id', 'petal-length', 'petal-width',…